In [26]:
#의약품 id 리스트 생성
import requests
from bs4 import BeautifulSoup

page_count = 314
all_contents_ids = []  

for page_num in range(1, page_count + 1):
    url = f"https://terms.naver.com/list.naver?cid=67992&categoryId=67992&page={page_num}"
    response = requests.get(url)
    
    soup = BeautifulSoup(response.text, 'html.parser')

    contents_id_elements = soup.find_all(class_="__TLIST_PUT")

    # data-contentsid 값을 추출함!
    contents_ids = [element.get("data-contentsid") for element in contents_id_elements]
    all_contents_ids.extend(contents_ids)
    
print(all_contents_ids)

['6552938', '6555088', '6551091', '6554393', '6551038', '6555177', '6553028', '6551644', '6551448', '6554973', '6553067', '6554658', '6554093', '6553477', '6551399', '6551152', '6554829', '6552064', '6551097', '6551974', '6553425', '6553699', '6553826', '6551913', '6554168', '6550852', '6552482', '6552616', '6554834', '6554645', '6554113', '6552486', '6682682', '6551317', '6552320', '6552906', '6552213', '6551909', '6554910', '6552476', '6554421', '6554690', '6551300', '6552881', '6553140', '6688729', '6552813', '6551055', '6551049', '6554617', '6551627', '6551720', '6554598', '6551408', '6551660', '6553587', '6550802', '6550976', '6551515', '6551223', '6553125', '6554074', '6553701', '6553979', '6551339', '6553098', '6554053', '6550983', '6550777', '6692537', '6551765', '6692505', '6550793', '6551135', '6554092', '6553682', '6551712', '6551791', '6553481', '6666449', '6552053', '6552313', '6553630', '6551684', '6694351', '6552176', '6552122', '6552128', '6553073', '6551905', '6554304'

In [27]:
all_contents_ids.__len__()

4696

In [28]:
import os
import csv
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

def extract_medicine_name(soup):
    medicine_name_tag = soup.find("h2", class_="headword")
    if medicine_name_tag:
        medicine_name = medicine_name_tag.text.strip().replace(' 개요정보', '')
        return medicine_name
    return None

def extract_image_url(soup):
    image_tag = soup.find('img', class_='lazyLoadImage')
    if image_tag:
        image_url = image_tag.get('data-src')
        return image_url
    return None

def download_image(image_url, file_path):
    response = requests.get(image_url, stream=True)
    if response.status_code == 200:
        total_size = int(response.headers.get('content-length', 0))
        os.makedirs(os.path.dirname(file_path), exist_ok=True)  # 폴더 생성
        with open(file_path, 'wb') as file, tqdm(
            desc=file_path,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as progress_bar:
            for data in response.iter_content(chunk_size=1024):
                file.write(data)
                progress_bar.update(len(data))
        print(f"이미지 다운로드 완료: {file_path}")
    else:
        print("이미지 다운로드 실패")

def scrape_and_save_images(base_url, all_contents_ids):
    session = requests.Session()  # 세션 유지
    for content_id in tqdm(all_contents_ids, desc="진행 중", unit="개"):  # tqdm 적용하여 전체 진행률 표시
        url = f"{base_url}&docId={content_id}"
        response = session.get(url)  # 세션을 통한 요청
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 대기 시간 추가
        time.sleep(1)

        # 의약품명 추가
        medicine_name = extract_medicine_name(soup)
        if not medicine_name:
            print("의약품명을 찾을 수 없습니다. 다음으로 넘어갑니다.")
            continue

        # 이미지 URL 가져오기
        image_url = extract_image_url(soup)
        if not image_url:
            print("이미지 URL을 찾을 수 없습니다. 다음으로 넘어갑니다.")
            continue

        # 이미지를 다운로드할 폴더 생성
        folder_name = "medicine_images"
        image_file_name = f"{medicine_name}.jpg"
        image_file_path = os.path.join(folder_name, image_file_name)
        
        # 이미지 다운로드
        download_image(image_url, image_file_path)

        # CSV 파일로 저장
        csv_file_name = f"{medicine_name}.csv"
        csv_file_path = os.path.join(folder_name, csv_file_name)
        with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['의약품명', '이미지 URL'])
            writer.writerow([medicine_name, image_url])
        print(f"CSV 파일 저장 완료: {csv_file_path}")

# 메인 URL
main_url = "https://terms.naver.com/entry.naver?cid=67992&categoryId=67992"

scrape_and_save_images(main_url, all_contents_ids)


진행 중:   0%|                                                                          | 0/4696 [00:00<?, ?개/s]
medicine_images/페니라민정(클로르페니라민말레산염).jpg: 100%|███████████████| 3.79k/3.79k [00:00<00:00, 8.88MB/s]
진행 중:   0%|                                                                | 1/4696 [00:01<2:13:21,  1.70s/개]

이미지 다운로드 완료: medicine_images/페니라민정(클로르페니라민말레산염).jpg
CSV 파일 저장 완료: medicine_images/페니라민정(클로르페니라민말레산염).csv



medicine_images/포리부틴정(트리메부틴말레산염).jpg: 100%|███████████████████| 4.89k/4.89k [00:00<00:00, 5.94MB/s]
진행 중:   0%|                                                                | 2/4696 [00:03<2:02:52,  1.57s/개]

이미지 다운로드 완료: medicine_images/포리부틴정(트리메부틴말레산염).jpg
CSV 파일 저장 완료: medicine_images/포리부틴정(트리메부틴말레산염).csv



medicine_images/지르텍정(세티리진염산염).jpg: 100%|█████████████████████████| 3.81k/3.81k [00:00<00:00, 9.30MB/s]
진행 중:   0%|                                                                | 3/4696 [00:04<1:58:29,  1.51s/개]

...

이미지 다운로드 완료: medicine_images/유디비타연질캡슐.jpg
CSV 파일 저장 완료: medicine_images/유디비타연질캡슐.csv



medicine_images/난타코연질캡슐.jpg: 100%|███████████████████████████████████| 5.25k/5.25k [00:00<00:00, 4.70MB/s]
진행 중: 100%|████████████████████████████████████████████████████████████▉| 4695/4696 [2:06:33<00:01,  1.64s/개]

이미지 다운로드 완료: medicine_images/난타코연질캡슐.jpg
CSV 파일 저장 완료: medicine_images/난타코연질캡슐.csv



medicine_images/디페나엘플라스타(디클로페낙디에틸암모늄).jpg: 100%|█████████| 1.00k/1.00k [00:00<00:00, 3.40MB/s]
진행 중: 100%|█████████████████████████████████████████████████████████████| 4696/4696 [2:06:34<00:00,  1.62s/개]

이미지 다운로드 완료: medicine_images/디페나엘플라스타(디클로페낙디에틸암모늄).jpg
CSV 파일 저장 완료: medicine_images/디페나엘플라스타(디클로페낙디에틸암모늄).csv
